# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Необходимо построить модель с предельно большим значением *F1*-меры не менее 0.59.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

Загрузим необходимые функции и библиотеки. Будем добавлять их сюда по мере необходимости.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

Прочтем данные.

In [2]:
bank = pd.read_csv('/datasets/Churn.csv')
bank.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [3]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Переведем названия столбцов в нижний регистр.

In [4]:
bank.columns = bank.columns.str.lower()

In [5]:
bank['geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

Удалим столбец с фамилией, т.к. есть ID.

In [6]:
bank = bank.drop(['surname'], axis=1)

Индекс строки и ID клиента никак не влияют на целевой признак, их тоже убираем. 

In [7]:
bank = bank.drop(['rownumber', 'customerid'], axis=1)

Проведем прямое кодирование техникой OHE. Чтобы избежать избытка данных уберём по одному столбцу(drop_first=True)

In [8]:
bank = pd.get_dummies(bank, drop_first=True)
bank.head(10)

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8.0,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7.0,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4.0,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4.0,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2.0,134603.88,1,1,1,71725.73,0,0,0,1


In [9]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
creditscore          10000 non-null int64
age                  10000 non-null int64
tenure               9091 non-null float64
balance              10000 non-null float64
numofproducts        10000 non-null int64
hascrcard            10000 non-null int64
isactivemember       10000 non-null int64
estimatedsalary      10000 non-null float64
exited               10000 non-null int64
geography_Germany    10000 non-null uint8
geography_Spain      10000 non-null uint8
gender_Male          10000 non-null uint8
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


Столбец tenure содержит 10% NaN. Заполним пропуски медианным значением, иначе модель будет выдавать ошибку. Скорее всего эти данные неизвестны, поэтому и не внесены в базу.

In [10]:
bank['tenure'] = bank['tenure'].fillna(bank['tenure'].median())

Целевой признак - 'exited'. Разобъем выборку на тренировочную, валидационную и тестовую в соотношении 60:20:20

In [11]:
target = bank['exited']
features = bank.drop('exited', axis=1)

features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

Произведем масштабирование числовых признаков.

In [12]:
numeric = ['creditscore', 'age', 'tenure', 'balance','estimatedsalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[ numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric]= scaler.transform(features_test[numeric])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [13]:
features_train.head()

,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,geography_Germany,geography_Spain,gender_Male
7479,-0.886751,-0.373192,1.082277,1.232271,1,1,0,-0.187705,0,1,1
3411,0.608663,-0.183385,1.082277,0.600563,1,0,0,-0.333945,0,0,0
6027,2.052152,0.480939,-0.737696,1.027098,2,0,1,1.503095,1,0,1
1247,-1.457915,-1.417129,0.354288,-1.233163,2,1,0,-1.071061,0,0,1
3716,0.130961,-1.132419,-1.101690,1.140475,1,0,0,1.524268,1,0,0


# 2. Исследование задачи

Проверим баланс классов

In [14]:
1 - target.mean()

0.7963

Соотношение классов 80:20, до баланса далеко. Но 20% ушедших из банка!!!  
Обучим модель дерева решений.

In [15]:
def f1_auc(model, features, target):
    model.fit(features, target)
    predicted_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('F1 -', f1_score(target_valid, predicted_valid))
    print('AUC-ROC -', roc_auc_score(target_valid, probabilities_one_valid))

model = DecisionTreeClassifier(random_state=12345)
f1_auc(model, features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Accuracy -', accuracy_score(target_valid, predicted_valid))

F1 - 0.5006045949214026
AUC-ROC - 0.6837644190927842
Accuracy - 0.7935


Модель предсказывает даже хуже, чем константная (как если бы все значения заполнили нулями).
Посмотрим, что покажет случайный лес.

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=10)
f1_auc(model, features_train, target_train)

F1 - 0.5522388059701492
AUC-ROC - 0.8152896823716572


У леса значения получше. Теперь регрессия.

In [17]:
model = LogisticRegression(random_state=12345, solver='liblinear')
f1_auc(model, features_train, target_train)

F1 - 0.33108108108108103
AUC-ROC - 0.7586620412656742


У регрессии значение F1 ниже, чем у дерева, но при этом AUC-ROC - выше. Получается, что регрессия в данном случае лучше предсказывает, но при этом хуже охватывает нашу выборку с точки зрения полноты и точности.

# 3. Борьба с дисбалансом

Проведем борьбу с дисбалансом с помощью взвешивания классов (укажем параметр ,class_weight='balanced'). Заодно для леса поменяем в циклах количество деревьев и глубину и получим максимальное значение F1. Дерево обучать не будем: лес - это много деревьев и он должен показывать лучшие результаты.

In [18]:
f1_max = 0
for estim in range(10, 101, 10):
    for depth in range(1, 16, 1):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        f1 = f1_score(target_valid, predicted_valid)
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if f1 > f1_max:
            f1_max = f1
            est = estim
            dep = depth
            auc = auc_roc
print('Наибольшее значение F1 -', f1_max)
print('AUC-ROC =', auc, 'при:')
print('max_depth =', dep)
print('n_estimators =', est)

Наибольшее значение F1 - 0.6337543053960963
AUC-ROC = 0.8570445623310086 при:
max_depth = 9
n_estimators = 80


In [19]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
f1_auc(model, features_train, target_train)

F1 - 0.4888507718696398
AUC-ROC - 0.7636962478601975


Случайный лес показал хороший результат, в принципе задача решена, F1 > 0.59  
Логистическая регрессия тоже улучшила показатели.  
Попробуем другой способ - увеличение выборки. Уравняем классы, увеличив количество единиц в 4 раза (изначально соотношение нулей и единиц 4 к 1)

In [20]:
def upsample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * 4)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * 4)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train)

In [21]:
f1_max = 0
for estim in range(10, 101, 10):
    for depth in range(1, 16, 1):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        f1 = f1_score(target_valid, predicted_valid)
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if f1 > f1_max:
            f1_max = f1
            est = estim
            dep = depth
            auc = auc_roc
print('Наибольшее значение F1 -', f1_max)
print('AUC-ROC =', auc, 'при:')
print('max_depth =', dep)
print('n_estimators =', est)

Наибольшее значение F1 - 0.6303418803418803
AUC-ROC = 0.8523248991344008 при:
max_depth = 9
n_estimators = 90


In [22]:
model = LogisticRegression(random_state=12345, solver='liblinear')
f1_auc(model, features_upsampled, target_upsampled)

F1 - 0.4888507718696398
AUC-ROC - 0.7636493687960851


Тоже неплохой результат, хотя и чуть хуже, чем при взвешивании.
Попробуем наоборот уменьшить количество нулей в 4 раза.

In [23]:
def downsample(features, target):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=0.25, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=0.25, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train)

In [24]:
f1_max = 0
for estim in range(10, 101, 10):
    for depth in range(1, 16, 1):
        model = RandomForestClassifier(n_estimators=estim, max_depth=depth, random_state=12345, class_weight='balanced')
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        f1 = f1_score(target_valid, predicted_valid)
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if f1 > f1_max:
            f1_max = f1
            est = estim
            dep = depth
            auc = auc_roc
print('Наибольшее значение F1 -', f1_max)
print('AUC-ROC =', auc, 'при:')
print('max_depth =', dep)
print('n_estimators =', est)

Наибольшее значение F1 - 0.61
AUC-ROC = 0.8357259298689201 при:
max_depth = 4
n_estimators = 10


In [25]:
model = LogisticRegression(random_state=12345, solver='liblinear')
f1_auc(model, features_downsampled, target_downsampled)

F1 - 0.48406546080964685
AUC-ROC - 0.7622823147974522


Значения метрик F1 и AUC-ROC чуть снизились. Меньше данных - хуже обучается модель.  
Сведем результаты в таблицу.

In [26]:
data = [[0.5522, 0.3311, 0.6338, 0.4889, 0.6303, 0.4889, 0.61, 0.4841], 
        [0.8153, 0.7587, 0.8570, 0.7637, 0.8523, 0.7636, 0.8357, 0.7623]]
table = pd.DataFrame(data, index=['F1', 'AUC-ROC'], 
        columns=['forest', 'regress', 'for_bal', 'regr_bal', 'for_up', 'regr_up', 'for_down', 'regr_down'])
table

,forest,regress,for_bal,regr_bal,for_up,regr_up,for_down,regr_down
F1,0.5522,0.3311,0.6338,0.4889,0.6303,0.4889,0.6100,0.4841
AUC-ROC,0.8153,0.7587,0.8570,0.7637,0.8523,0.7636,0.8357,0.7623


Видим, что баланс классов важен, при этом увеличиваются обе метрики, причем при любом из выбранных способов борьбы с дисбалансом.  
Итак, по результатам валидации вперед вышла модель "Случайный лес" со взвешенными классами и параметрами n_estimators=80, max_depth=9.

# 4. Тестирование модели

Проверим нашу лучшую модель на тестовой выборке.

In [27]:
model = RandomForestClassifier(n_estimators=80, max_depth=9, random_state=12345,class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)

f1 = f1_score(target_test, predicted_test)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)

print('Значение F1 -', f1)
print('Значение AUC-ROC -', auc_roc)

Значение F1 - 0.6124293785310734
Значение AUC-ROC - 0.8571696266214541


Итак, наилучшая модель проверена на тестовой выборке. Значение F1 укладывается в заданный диапазон. Значение метрики AUC-ROC - 0.86, что значительно превышает значение AUC-ROC случайной модели (0.5)